<a href="https://colab.research.google.com/github/AwkwaBear/EE-628/blob/main/sg13g2_LDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# install conda.
CONDA_PREFIX = os.environ.get('CONDA_PREFIX', None)
if not CONDA_PREFIX:
  !python -m pip install condacolab
  import condacolab
  condacolab.install()

# install sky130a
!conda install -c litex-hub open_pdks.sky130a

# install ngspice
!conda install -c conda-forge ngspice

# Install Pytorch with cpu-only option.
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 cpuonly -c pytorch

# Check the version of Python (should be larger than 3.7) and Pytorch.
import torch
import sys
print("Python version")
print (sys.version)

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

# Install PyG to enable graph neural networks. We need to manually guide Colab to install the correct version of PyG based on which Pytorch version we have installed
# (https://colab.research.google.com/drive/17VFsRZeZa7rgtrQ39-RMiYIE0ePrSlgN?authuser=1#scrollTo=rFwf_0DrvXLv). In this case it is 1.13.1 with cpu-only option.
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-geometric

# Check the version of PyG.
import torch_geometric
print(torch_geometric.__version__)

# Install Gymnasium for the RL environment.
!pip install gymnasium

# Check Gymnasium version.
import gymnasium as gym
print(gym.__version__)

# Install tabulate for showing simulation results nicely during the runtime later.
!pip install tabulate

# Similarly, check tabulate version.
import tabulate
print(tabulate.__version__)